# Python Data Analysis for Internal Radiotherapy Applications

## Contents:
* CT visualisation
* Dose map rescaling
* Dose map resampling
* CT and Dose map overlay
* Dosimetry

http://insightsoftwareconsortium.github.io/SimpleITK-Notebooks/

# Exercice : dose map analysis

In [1]:
import SimpleITK as sitk

# Read a sitk image
img_ct = sitk.ReadImage('Ex_dose3D/data/patient-2mm.mhd')
# img_ct = sitk.ReadImage('Ex_InternalRT/data/patient_CT.mhd')
print('Image size: ', img_ct.GetSize())
print('Image spacing: ', img_ct.GetSpacing())
print('Image origin: ', img_ct.GetOrigin())

Image size:  (126, 126, 111)
Image spacing:  (2.0, 2.0, 2.0)
Image origin:  (0.0, 0.0, 0.0)


In [2]:
# Convert sitk image to a numpy array
arr_ct = sitk.GetArrayFromImage(img_ct)
print('Array size: ', arr_ct.shape, ' <--- be careful to the dimension order!)')

Array size:  (111, 126, 126)  <--- be careful to the dimension order!)


In [5]:
import matplotlib.pyplot as plt
%matplotlib inline
from ipywidgets import interact
# to enable interactive widgets in jupyter notebook run this command in a terminal:
#jupyter nbextension enable --py widgetsnbextension

def show_ct(sx,sy,sz):
    fig, ax = plt.subplots(1,3, figsize=(10, 5))
    ax[0].imshow(arr_ct[sx,:,:], origin='lower', cmap=plt.cm.gray)
    ax[1].imshow(arr_ct[:,sy,:], cmap=plt.cm.gray)
    ax[2].imshow(arr_ct[:,:,sz], cmap=plt.cm.gray)
    
interact(show_ct, sx=(0,arr_ct.shape[0]-1), sy=(0,arr_ct.shape[1]-1), sz=(0,arr_ct.shape[2]-1));

interactive(children=(IntSlider(value=55, description='sx', max=110), IntSlider(value=62, description='sy', ma…

# Run the GATE example and analyse the output
## some high statistic results with 1e6 particles are available in results/

In [6]:
import SimpleITK as sitk
import numpy as np

img_dose = sitk.ReadImage('Ex_dose3D/results/3d-pat-Dose.mhd')
# img_dose = sitk.ReadImage('Ex_InternalRT/results/out-Dose.mhd')
arr_dose = sitk.GetArrayFromImage(img_dose)
print('Image size = ', arr_dose.shape)
print('Image min and max: ',  np.amin(arr_dose), np.amax(arr_dose))

Image size =  (50, 50, 50)
Image min and max:  0.0 1.23685895e-05


In [7]:
filter = sitk.RescaleIntensityImageFilter()
filter.SetOutputMaximum(1.0)
filter.SetOutputMinimum(0.0)
img_dose = filter.Execute(img_dose)

In [8]:
arr_dose = sitk.GetArrayFromImage(img_dose)
print('Image size = ', arr_dose.shape)
print('Image min and max: ',  np.amin(arr_dose), np.amax(arr_dose))

Image size =  (50, 50, 50)
Image min and max:  0.0 1.0


In [11]:
import matplotlib.pyplot as plt
%matplotlib inline
from ipywidgets import interact

def show_dose(nslice):
    plt.imshow(arr_dose[:,:,nslice], cmap=plt.cm.hot)
    plt.colorbar()
    
interact(show_dose, nslice=(0,len(arr_dose)-1));

interactive(children=(IntSlider(value=24, description='nslice', max=49), Output()), _dom_classes=('widget-inte…

In [12]:
img_resampled_dose = sitk.Resample(img_dose, img_ct, sitk.Transform(), sitk.sitkLinear, 0)

In [13]:
arr_resampled_dose = sitk.GetArrayFromImage(img_resampled_dose)
print('Image size = ', arr_resampled_dose.shape)
print('Image min and max: ',  np.amin(arr_resampled_dose), np.amax(arr_resampled_dose))

Image size =  (111, 126, 126)
Image min and max:  0.0 0.976139


In [17]:
def show_fusion(nslice=56, opacity=0.7):
    fig, ax = plt.subplots(figsize=(10, 6))
    plt.imshow(arr_ct[:,:,nslice], cmap=plt.cm.gray)
    a = arr_resampled_dose[:,:,nslice]
    b = np.ma.masked_where(a <= 0.001, a)
    plt.imshow(b, alpha=opacity, cmap=plt.cm.hot)
    plt.colorbar()
    
interact(show_fusion, nslice=(0,arr_ct.shape[2]-1), opacity=(0,1,0.1));

interactive(children=(IntSlider(value=56, description='nslice', max=125), FloatSlider(value=0.7, description='…

Questions:
- plot a depth dose profile
- display uncertainty map
